# Gather Bachelor students data

The purpose of this script is only to collect the data of Bachelor students, group and clean it all, serialize it in order to be used by other scripts analyzing the dataset.

## Assignment

1) Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both Bachelor semestre 1 and Bachelor semestre 6.

In [2]:
import requests as rq
import pandas as pd
import bs4

In [3]:
ISA_url = "http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247"
ISA_form_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247"

### Find the parameters values

In order to find all the existing parameter names and possible values, we make a request on the form page and then parse it

In [4]:
r = rq.get(ISA_form_url)
bs = bs4.BeautifulSoup(r.content, "html.parser")

We parse the HTML file looking for "select" tags which correspond to the different menus, and for each of them we look at the "option" tags which are the entries. We save our results in a dictionnary

In [103]:
params = pd.DataFrame(columns=['Parameter','Name','Value'])

for sel in bs.find_all('select'):
    p_name = sel['name'] #if (sel['name']) else '' # Assign select tag name and if none, assign empty string

    for opt in sel.find_all('option'):
        o_name = opt.get_text() #if (opt.get_text()) else ''
        value = opt['value']
        tmp = pd.DataFrame([[p_name, o_name, value]], columns=['Parameter', 'Name', 'Value'])
        params = pd.concat([params, tmp])
        
params.set_index(['Parameter', 'Name'])

Value
Parameter           Name                                                 
ww_x_UNITE_ACAD                                                      null
                    Architecture                                   942293
                    Chimie et génie chimique                       246696
                    Cours de mathématiques spéciales               943282
                    EME (EPFL Middle East)                      637841336
                    Génie civil                                    942623
                    Génie mécanique                                944263
                    Génie électrique et électronique               943936
                    Humanités digitales                        2054839157
                    Informatique                                   249847
                    Ingénierie financière                       120623110
                    Management de la technologie                   946882
                    Mathématiques                                  944590
                    Microtechnique                                 945244
                    Physique                                       945571
                    Science et génie des matériaux                 944917
                    Sciences et ingénierie de l'environnement      942953
                    Sciences et technologies du vivant             945901
                    Section FCUE                               1574548993
                    Systèmes de communication                      946228
ww_x_PERIODE_ACAD                                                    null
                    2016-2017                                   355925344
                    2015-2016                                   213638028
                    2014-2015                                   213637922
                    2013-2014                                   213637754
                    2012-2013                                   123456101
                    2011-2012                                   123455150
                    2010-2011                                    39486325
                    2009-2010                                      978195
                    2008-2009                                      978187
                    2007-2008                                      978181
ww_x_PERIODE_PEDAGO                                                  null
                    Bachelor semestre 1                            249108
                    Bachelor semestre 2                            249114
                    Bachelor semestre 3                            942155
                    Bachelor semestre 4                            942163
                    Bachelor semestre 5                            942120
                    Bachelor semestre 5b                          2226768
                    Bachelor semestre 6                            942175
                    Bachelor semestre 6b                          2226785
                    Master semestre 1                             2230106
                    Master semestre 2                              942192
                    Master semestre 3                             2230128
                    Master semestre 4                             2230140
                    Mineur semestre 1                             2335667
                    Mineur semestre 2                             2335676
                    Mise à niveau                              2063602308
                    Projet Master automne                          249127
                    Projet Master printemps                       3781783
                    Semestre automne                               953159
                    Semestre printemps                            2754553
                    Stage automne 3ème année                       953137
                    Stage automne 4ème année                      222661

In this homework we are interesting in students from the "Informatique" section, and we want the data as excel files. Therefore we give the following parameters fixed values:

In [7]:
ww_x_UNITE_ACAD = params['ww_x_UNITE_ACAD']['Informatique']
ww_i_reportModel = '133685247'  # Given from the base url
ww_i_reportModelXls = '133685271'  # Request XLS files, given by Postman

### Gather the data